In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)

2.3.0


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


## Data from Kaggle

https://www.kaggle.com/namanj27/ner-dataset


In [ ]:
raw_data = pd.read_csv('/content/drive/My Drive/dat390_data/ner_datasetreference.csv', encoding = "ISO-8859-1")
raw_data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [ ]:
data = raw_data
from itertools import chain
tok2idx = {}
idx2tok = {}
vocab = list(set(data['Word'].to_list()))
idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
data['Word_idx'] = data['Word'].map(token2idx)


tok2idx = {}
idx2tok = {}
tags = list(set(data['Tag'].to_list()))
idx2tok = {idx:tok for  idx, tok in enumerate(tags)}
tok2idx = {tok:idx for  idx, tok in enumerate(tags)}
data['Tag_idx'] = data['Tag'].map(tag2idx)

In [ ]:
data.head()

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,Thousands,NNS,O,32834,11
1,NaN,of,IN,O,23230,11
2,NaN,demonstrators,NNS,O,1269,11
3,NaN,have,VBP,O,28447,11
4,NaN,marched,VBN,O,26972,11


In [ ]:
n_tags = len(tags)
n_vocab = len(vocab)
n_tags, n_vocab

(17, 35178)

In [ ]:
print(tags)

['I-eve', 'I-tim', 'I-geo', 'I-org', 'B-tim', 'I-art', 'I-gpe', 'I-nat', 'B-geo', 'B-per', 'B-gpe', 'O', 'I-per', 'B-nat', 'B-eve', 'B-art', 'B-org']


In [ ]:
data_fillna = data.fillna(method='ffill', axis=0)
data_sentences = data_fillna.groupby(['Sentence #'], as_index=False
                                )['Word', 'POS', 'Tag', 'Word_idx', 
                                  'Tag_idx'].agg(lambda x: list(x))

In [ ]:
' '.join(data_sentences.iloc[0]['Word'])

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [ ]:
inputs = np.array(data_sentences['Word_idx'])
targets = np.array(data_sentences['Tag_idx'])
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, padding='post', truncating='post', value= tag2idx["O"])

In [ ]:
input_len = len(inputs[0])

In [ ]:
from sklearn.model_selection import train_test_split
train_inputs, test_inputs, train_targets, test_targets = train_test_split(inputs, targets, test_size=0.1, random_state=42)

In [ ]:
train_inputs = np.array(train_inputs).reshape(len(train_inputs), 104, 1)
train_targets = np.array(train_targets)
test_inputs = np.array(test_inputs).reshape(len(test_inputs), 104, 1)
test_targets = np.array(test_targets)

## Building LSTM network

In [ ]:
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
tf.random.set_seed(2)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=n_vocab+1, output_dim=32))
model.add(Bidirectional(LSTM(units=32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))
model.add(LSTM(units=32, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
model.add(TimeDistributed(Dense(n_tags, activation="relu")))
model.compile(loss='SparseCategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 32)          1125728   
_________________________________________________________________
bidirectional_8 (Bidirection (None, None, 64)          16640     
_________________________________________________________________
lstm_16 (LSTM)               (None, None, 32)          12416     
_________________________________________________________________
time_distributed_7 (TimeDist (None, None, 17)          561       
Total params: 1,155,345
Trainable params: 1,155,345
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(train_inputs, train_targets, batch_size=500, epochs=5, validation_split=0.1)

Epoch 1/5
78/78 [==============================] - 101s 1s/step - loss: 0.5618 - accuracy: 0.9470 - val_loss: 0.3137 - val_accuracy: 0.9676
Epoch 2/5
78/78 [==============================] - 99s 1s/step - loss: 0.2900 - accuracy: 0.9678 - val_loss: 0.2519 - val_accuracy: 0.9676
Epoch 3/5
78/78 [==============================] - 98s 1s/step - loss: 0.2557 - accuracy: 0.9679 - val_loss: 0.2374 - val_accuracy: 0.9676
Epoch 4/5
78/78 [==============================] - 99s 1s/step - loss: 0.2548 - accuracy: 0.9670 - val_loss: 0.2566 - val_accuracy: 0.9651
Epoch 5/5
78/78 [==============================] - 98s 1s/step - loss: 0.2172 - accuracy: 0.9676 - val_loss: 0.2243 - val_accuracy: 0.9677


In [ ]:
model.evaluate(test_inputs, test_targets)

150/150 [==============================] - 5s 35ms/step - loss: 0.2205 - accuracy: 0.9681


[0.2204880714416504, 0.9681445360183716]

## Refereences:

https://github.com/snehalnair/Named-Entity-Recognition/

https://www.kaggle.com/namanj27/ner-dataset
